# <font color='#6629b2'>Predicting sentiment ratings with RNN using Keras</font>


## <font color='#6629b2'>Study Material</font>
- https://web.stanford.edu/~jurafsky/slp3/9.pdf
- https://www.youtube.com/watch?v=UNmqTiOnRfg
- https://www.youtube.com/watch?v=WCUNPb-5EYI
- https://www.youtube.com/watch?v=OuYtk9Ymut4

## <font color='#6629b2'>Dataset</font>

The Large Movie Review Datasetconsists of 50,000 movie reviews from [IMDB](http://www.imdb.com/). The ratings are on a 1-10 scale, but the dataset specifically contains "polarized" reviews: positive reviews with a rating of 7 or higher, and negative reviews with a rating of 4 or lower. There are an equal number of positive and negative reviews. 

You can download the dataset from: https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [1]:
'''Load the dataset into variable "reviews". You can truncate the dataset to keep a few hundred records if it's
    taking too long to process/train. Keep in mind, bigger the dataset, higher the accuracy score!
'''
import pandas as pd
import numpy as np



reviews = pd.read_csv("IMDB Dataset.csv").iloc[:1000,:]


reviews

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
995,Nothing is sacred. Just ask Ernie Fosselius. T...,positive
996,I hated it. I hate self-aware pretentious inan...,negative
997,I usually try to be professional and construct...,negative
998,If you like me is going to see this in a film ...,negative


In [2]:
#Split the data into train and split

from sklearn.model_selection import train_test_split

X = reviews.iloc[:, 0].values
Y= reviews.iloc[:, 1].values

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.30, random_state = 0) 

In [3]:
# convert the above train test datasets into a pandas dataframe. You shoud have 4 dataframes.
# Name them train_reviews, test_reviews, train_sentiment, test_sentiment.
# In train_sentiment and test_sentiment, convert "positive" to 1 and "negative" to 0

train_reviews = pd.DataFrame(X_train, columns = ["review"])
test_reviews = pd.DataFrame(X_test, columns = ["review"])

train_sentiment = pd.DataFrame(y_train, columns = ["sentiment"])
train_sentiment.sentiment.replace(('positive','negative'),(1,0), inplace = True)

test_sentiment = pd.DataFrame(y_test, columns = ["sentiment"])
test_sentiment.sentiment.replace(('positive','negative'),(1,0), inplace = True)

test_sentiment

,sentiment
0,0
1,0
2,1
3,0
4,0
...,...
295,0
296,0
297,1
298,1


In [4]:
train_reviews.head()

,review
0,Marion Davies stars in this remarkable comedy ...
1,I'm not sure why the producers needed to trade...
2,Joseph Brady and Clarence Doolittle are two sa...
3,This movie was a fairly entertaining comedy ab...
4,"Yes, indeed, it could have been a good movie. ..."


## <font color='#6629b2'>Preparing the data</font>

###  <font color='#6629b2'>Tokenization</font>

The first preprocessing step is to tokenize each of the reviews into (lowercased) individual words, since the models will encode the reviews at the word level (rather than subword units like characters, for example). For this we'll use [spaCy](https://spacy.io/), which is a fast and extremely user-friendly library that performs various language processing tasks. Once you load a spaCy model for a particular language, you can provide any text as input to the model, e.g. encoder(text) and access its linguistic features.

In [5]:
'''Lowercase and tokenise all the reviews in train_reviews using spacy'''

import spacy
encoder = spacy.load('en_core_web_sm')

def text_to_tokens(text_seqs):
    #complete this function that lowers and tokenizes the reviews
    tokens = []
    for seq in text_seqs:
        token = encoder(seq)
        tokens.append([(tks.text).lower() for tks in token])
    return tokens    


train_reviews['Tokenized_Review'] = text_to_tokens(train_reviews['review'])
train_reviews[['review','Tokenized_Review']][:10]


,review,Tokenized_Review
0,Marion Davies stars in this remarkable comedy ...,"[marion, davies, stars, in, this, remarkable, ..."
1,I'm not sure why the producers needed to trade...,"[i, 'm, not, sure, why, the, producers, needed..."
2,Joseph Brady and Clarence Doolittle are two sa...,"[joseph, brady, and, clarence, doolittle, are,..."
3,This movie was a fairly entertaining comedy ab...,"[this, movie, was, a, fairly, entertaining, co..."
4,"Yes, indeed, it could have been a good movie. ...","[yes, ,, indeed, ,, it, could, have, been, a, ..."
5,Black comedy isn't always an easy sell. Every ...,"[black, comedy, is, n't, always, an, easy, sel..."
6,I thought before starting with these movie tha...,"[i, thought, before, starting, with, these, mo..."
7,If you've read Mother Night and enjoyed it so ...,"[if, you, 've, read, mother, night, and, enjoy..."
8,"This is definitely a ""must see"" film. The exce...","[this, is, definitely, a, "", must, see, "", fil..."
9,"This movie is among my favorite foreign films,...","[this, movie, is, among, my, favorite, foreign..."


###  <font color='#6629b2'>Lexicon</font>

Then we need to assemble a lexicon (aka vocabulary) of words that the model needs to know. Each tokenized word in the reviews is added to the lexicon, and then each word is mapped to a numerical index that can be read by the model. Since large datasets may contain a huge number of unique words, it's common to filter all words occurring less than a certain number of times, and replace them with some generic &lt;UNK&gt; token. The min_freq parameter in the function below defines this threshold. When assigning the indices, the number 1 will represent unknown words. The number 0 will represent "empty" word slots, which is explained below. Therefore "real" words will have indices of 2 or higher.

In [6]:
'''Count tokens (words) in texts and add them to the lexicon'''


import pickle
import gensim
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary

def make_lexicon(token_seqs, min_freq=1, use_padding=False):
    #     First, count how often each word appears in the text. Save this count in a dictionary called token_counts
    token_counts = {}
    for seq in token_seqs:
        for token in seq:
            if token in token_counts:
                token_counts[token] += 1
            else:
                token_counts[token] = 1                  
            
    
    # Then, assign each word to a numerical index, i.e save  all these words in a list. Filter words that occur less than or equal to min_freq times.
#     lexicon = [key for key,value in token_counts.items() if value>min_freq]
    lexicon = [key for key,value in token_counts.items() if value>min_freq]
    
    
    #create a dictionary lexicon that maps each word to its index. Note that indexes will start from 2,  index 0 is saved for padding and index 1 for unknown words ('<UNK>')
    lexicon = {val: key + 2 for key, val in enumerate(lexicon)} 
    
    lexicon[u'<UNK>'] = 1 # Unknown words are those that occur fewer than min_freq times
    lexicon_size = len(lexicon)
    print(lexicon)

    print("LEXICON SAMPLE ({} total items):".format(lexicon_size))
    print(dict(list(lexicon.items())[:20]))
    
    return lexicon

lexicon = make_lexicon(token_seqs=train_reviews['Tokenized_Review'], min_freq=1)

{'marion': 2, 'davies': 3, 'stars': 4, 'in': 5, 'this': 6, 'remarkable': 7, 'comedy': 8, '"': 9, 'show': 10, 'people': 11, 'released': 12, 'by': 13, 'mgm': 14, '1928': 15, '.': 16, 'plays': 17, 'a': 18, 'from': 19, ',': 20, 'georgia': 21, 'who': 22, 'arrives': 23, 'hollywood': 24, 'with': 25, 'her': 26, 'father': 27, '(': 28, ')': 29, 'the': 30, 'they': 31, 'arrive': 32, 'is': 33, 'hoot': 34, '-': 35, 'as': 36, 'outrageous': 37, 'southern': 38, 'costume': 39, 'lands': 40, 'job': 41, 'slapstick': 42, 'not': 43, 'what': 44, 'she': 45, 'wants': 46, 'but': 47, 'it': 48, 'brings': 49, 'success': 50, 'meets': 51, 'fellow': 52, 'star': 53, 'william': 54, 'haines': 55, 'immediately': 56, 'smitten': 57, 'well': 58, 'then': 59, 'gets': 60, 'at': 61, 'more': 62, 'studio': 63, 'high': 64, 'art': 65, 'studios': 66, 'and': 67, 'stuffy': 68, 'period': 69, 'pieces': 70, 'handsome': 71, 'fake': 72, 'actor': 73, 'shows': 74, 'ropes': 75, 'of': 76, 'how': 77, 'to': 78, 'be': 79, 'typical': 80, 'pretentio

###  <font color='#6629b2'>From strings to numbers</font>

Once the lexicon is built, we can use it to transform each review from a list of string tokens into a list of numerical indices.

In [7]:
'''Convert each review from a list of tokens to a list of numbers (indices)'''

def tokens_to_idxs(token_seqs, lexicon): 
    #complete this function to return a list of indexed tokens 
    idx_seqs = []
    for seq in token_seqs:
        tokens = []
        for token in seq:
#             print(token)
            if token in lexicon:
                tokens.append(lexicon[token])
            else:
                tokens.append(lexicon['<UNK>'])
        idx_seqs.append(tokens)
    return idx_seqs

train_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=train_reviews['Tokenized_Review'], lexicon=lexicon)
                                   
train_reviews[['Tokenized_Review', 'Review_Idxs']][:10]


,Tokenized_Review,Review_Idxs
0,"[marion, davies, stars, in, this, remarkable, ...","[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 9, 12, 13, 14..."
1,"[i, 'm, not, sure, why, the, producers, needed...","[127, 128, 43, 129, 130, 30, 131, 132, 78, 133..."
2,"[joseph, brady, and, clarence, doolittle, are,...","[218, 219, 67, 220, 1, 103, 221, 1, 20, 22, 17..."
3,"[this, movie, was, a, fairly, entertaining, co...","[6, 138, 159, 18, 380, 381, 8, 228, 382, 267, ..."
4,"[yes, ,, indeed, ,, it, could, have, been, a, ...","[415, 20, 416, 20, 48, 170, 176, 211, 18, 84, ..."
5,"[black, comedy, is, n't, always, an, easy, sel...","[455, 8, 33, 167, 456, 420, 457, 458, 16, 459,..."
6,"[i, thought, before, starting, with, these, mo...","[127, 619, 620, 621, 25, 622, 138, 88, 48, 623..."
7,"[if, you, 've, read, mother, night, and, enjoy...","[387, 302, 655, 656, 496, 546, 67, 614, 48, 16..."
8,"[this, is, definitely, a, "", must, see, "", fil...","[6, 33, 720, 18, 9, 721, 186, 9, 114, 16, 30, ..."
9,"[this, movie, is, among, my, favorite, foreign...","[6, 138, 33, 771, 684, 772, 773, 201, 20, 178,..."


In [8]:
'''Encode reviews (train_reviews['Review_Idxs']) as bag-of-words vectors'''

import numpy 

def idx_seqs_to_bows(idx_seqs, matrix_length):
    #complete the function to return an array having bag-of-words vectors of the encoded reviews
    # hint: numpy.bincount()
    blist = []
    blist = [np.bincount(seq, minlength = matrix_length) for seq in idx_seqs]
    for seq in idx_seqs:
        arr = np.array(blist)
       
        
    return arr
    


bow_train_reviews = idx_seqs_to_bows(train_reviews['Review_Idxs'], 
                                     matrix_length=len(lexicon) + 1) #add one to length for padding)

print("TRAIN INPUT:\n", bow_train_reviews)
print("SHAPE:", bow_train_reviews.shape, "\n")

#Showing an example mapping string words to counts
lexicon_lookup = {idx: lexicon_item for lexicon_item, idx in lexicon.items()}
lexicon_lookup[0] = ""
pd.DataFrame([(lexicon_lookup[idx], count) for idx, count in enumerate(bow_train_reviews[0])], 
                 columns=['Word', 'Count'])

TRAIN INPUT:
 [[ 0 12  2 ...  0  0  0]
 [ 0 16  0 ...  0  0  0]
 [ 0 16  0 ...  0  0  0]
 ...
 [ 0  9  0 ...  0  0  0]
 [ 0  8  0 ...  3  2  0]
 [ 0 17  0 ...  0  0  2]]
SHAPE: (700, 7765) 



,Word,Count
0,,0
1,<UNK>,12
2,marion,2
3,davies,7
4,stars,2
...,...,...
7760,dreyfuss,0
7761,goldblum,0
7762,byrne,0
7763,93,0


##  <font color='#6629b2'>Building a Recurrent Neural Network </font>



###  <font color='#6629b2'>Numerical lists to matrices</font>

The input representation for the RNN explicitly encodes the order of words in the review. We'll return to the lists of the word indices contained in train_reviews['Review_Idxs']. The input to the model will be these number sequences themselves. We need to put all the reviews in the training set into a single matrix, where each row is a review and each column is a word index in that sequence. This enables the model to process multiple sequences in parallel (batches) as opposed to one at a time. Using batches significantly speeds up training. However, each review has a different number of words, so we create a padded matrix equal to the length on the longest review in the training set. For all reviews with fewer words, we prepend the row with zeros representing an empty word position. This is why the number 0 was not assigned as a word index in the lexicon. We can tell Keras to ignore these zeros during training.

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def pad_idx_seqs(idx_seqs):
    
    #find the biggest review's length and save it in the variable below
    
#     max_seq_len = 0
#     for tk in idx_seqs:
#         if len(tk)>max_seq_len:
#             max_seq_len = len(tk)
    max_seq_len = max([len(x) for x in idx_seqs])
    
    #pad all these indexed reviews and return these padded sequences
    #HINT: use pad_sequences function by keras
    return np.array(pad_sequences(idx_seqs, maxlen=max_seq_len, padding='pre'))
    
    


train_padded_idxs = pad_idx_seqs(train_reviews['Review_Idxs'])

print("TRAIN INPUT:\n", train_padded_idxs)
print("SHAPE:", train_padded_idxs.shape, "\n")

TRAIN INPUT:
 [[  0   0   0 ...  18 114  16]
 [  0   0   0 ...   6 114  16]
 [  0   0   0 ... 378 379  16]
 ...
 [  0   0   0 ...   1  16  29]
 [  0   0   0 ... 138 763  16]
 [  0   0   0 ... 267   1  16]]
SHAPE: (700, 1446) 



###  <font color='#6629b2'>Model Layers</font>
The RNN will have four layers:

**1. Input**: The input layer takes in the matrix of word indices.

**2. Embedding**: A [layer](https://keras.io/layers/embeddings/) that converts integer word indices into distributed vector representations (embeddings), which were introduced above. The difference here is that rather than plugging in embeddings from a pretrained model as before, the word embeddings will be learned inside the model itself. Thus, the input to the model will be the word indices rather than their embeddings, and the embedding values will change as the model is trained. The mask_zero=True parameter in this layer indicates that values of 0 in the matrix (the padding) will be ignored by the model.

**3. GRU**: A [recurrent (GRU) hidden layer](https://keras.io/layers/recurrent/), the central component of the model. As it observes each word in the review, it integrates the word embedding representation with what it's observed so far to compute a representation (hidden state) of the review at that timepoint. There are a few architectures for this layer - we use the GRU variation, Keras also provides LSTM or just the simple vanilla recurrent layer (see the materials at the bottom for an explanation of the difference). This layer outputs the last hidden state of the sequence (i.e. the hidden representation of the review after its last word is observed).

**4. Dense**: An output [layer](https://keras.io/layers/core/#dense) that predicts the rating for the review based on its GRU representation given by the previous layer. It has one dimension that contains a continuous value (the rating). Add a proper activation function.

###  <font color='#6629b2'>Parameters</font>

Our function for creating the RNN takes the following parameters:

**n_input_nodes**: As with the standard bag-of-words MLP, this is the number of unique words in the lexicon, plus one to account for the padding represented by 0 values. This indicates the number of rows in the embedding layer, where each row corresponds to a word.

**n_embedding_nodes**: the number of dimensions (units) in the embedding layer, which can be freely defined. Here, it is set to 300.

**n_hidden_nodes**: the number of dimensions in the GRU hidden layer. Like the embedding layer, this can be freely chosen. Here, it is set to 500.

In [10]:
'''Create the model'''

import tensorflow as tf
import keras
from keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.layers import InputLayer

def create_rnn_model(n_input_nodes, n_embedding_nodes, n_hidden_nodes):
    
    #complete this function to create a model and compile it having the 4 layers listed above.
    #Note: Layer 1 -  Technically the shape of this layer is (batch_size, len(train_padded_idxs)).
    # However, both the batch size and the length of the input matrix can be inferred from the input at training time. 
    # The batch size is implicitly included in the shape of the input, so it does not need to 
    # be specified as a dimension of the input. None can be given as placeholder for the input matrix length.
    # By defining it as None, the model is flexible in accepting inputs with different lengths.
    
    model = keras.Sequential()
    
    input_layer = Input(shape=(None,))
    model.add(input_layer)
    
    # Layer 2
    model.add(layers.Embedding(input_dim=n_input_nodes, output_dim=n_embedding_nodes, mask_zero = True))
    # Layer 3
    model.add(layers.GRU(n_hidden_nodes, return_sequences=True))
    # Layer 4
    model.add(Dense(1, activation='sigmoid'))
    #Specify which layers are input and output, compile model with loss and optimization functions
    model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy'])
    return model


In [11]:
rnn_model = create_rnn_model(n_input_nodes=len(lexicon) + 1, n_embedding_nodes=300, n_hidden_nodes=500)

###  <font color='#6629b2'>Training</font>

In [12]:
'''
Train the rnn_model using the padded sequences and y=train_reviews['Rating'].
You need  to convert train_reviews['Rating'] to tensor before passing it as an argument
Hint: tf.convert_to_tensor
batch_size=20, epochs=5
'''
batch_size=20
epochs=5

rnn_model.fit(tf.convert_to_tensor(train_padded_idxs), tf.convert_to_tensor(train_sentiment['sentiment']), batch_size = batch_size, epochs = epochs)


Epoch 1/5


ValueError: in user code:

    File "C:\Users\Admin\anaconda3\lib\site-packages\keras\engine\training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Admin\anaconda3\lib\site-packages\keras\engine\training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Admin\anaconda3\lib\site-packages\keras\engine\training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Admin\anaconda3\lib\site-packages\keras\engine\training.py", line 809, in train_step
        loss = self.compiled_loss(
    File "C:\Users\Admin\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "C:\Users\Admin\anaconda3\lib\site-packages\keras\losses.py", line 141, in __call__
        losses = call_fn(y_true, y_pred)
    File "C:\Users\Admin\anaconda3\lib\site-packa

###  <font color='#6629b2'>Prediction</font>

In [13]:
'''Put test reviews in padded matrix just how we did for train_reviews'''

test_reviews['Tokenized_Review'] = text_to_tokens(test_reviews['review'])
lexicon1 = make_lexicon(token_seqs=test_reviews['Tokenized_Review'], min_freq=1)
test_reviews['Review_Idxs'] = tokens_to_idxs(token_seqs=test_reviews['Tokenized_Review'], lexicon=lexicon1)
test_padded_idxs = pad_idx_seqs(test_reviews['Review_Idxs'])


print("TEST INPUT:\n", test_padded_idxs)
print("SHAPE:", test_padded_idxs.shape, "\n")

{'tim': 2, 'robbins': 3, 'and': 4, 'john': 5, 'cusack': 6, 'are': 7, 'two': 8, 'actors': 9, 'i': 10, 'have': 11, 'appreciated': 12, 'throughout': 13, 'their': 14, 'careers': 15, ',': 16, 'that': 17, 'was': 18, 'the': 19, 'only': 20, 'reason': 21, 'for': 22, 'to': 23, 'watch': 24, 'this': 25, 'movie': 26, '.': 27, 'well': 28, 'all': 29, 'can': 30, 'say': 31, 'is': 32, 'totally': 33, 'it': 34, '!': 35, 'these': 36, 'great': 37, 'themselves': 38, 'way': 39, 'through': 40, 'by': 41, 'performing': 42, 'a': 43, 'number': 44, 'of': 45, 'irrelevant': 46, 'unimaginative': 47, 'extreme': 48, '(': 49, 'not': 50, 'bad': 51, 'on': 52, 'its': 53, 'supposed': 54, 'make': 55, 'people': 56, 'laugh': 57, 'but': 58, 'fail': 59, 'do': 60, 'so': 61, 'think': 62, 'director': 63, 'friend': 64, 'they': 65, 'decided': 66, 'support': 67, 'his': 68, 'starring': 69, 'in': 70, '-': 71, 'ca': 72, "n't": 73, 'anything': 74, 'else': 75, 'because': 76, 'cheap': 77, 'fortunately': 78, 'disappointed': 79, 'me': 80, 'eve

In [14]:
'''Predict the ratings '''

#Since ratings are integers, need to round predicted rating to nearest integer
test_reviews['RNN_Pred_Rating'] = numpy.round(rnn_model.predict(test_padded_idxs)[:,0]).astype(int)

###  <font color='#6629b2'>Evaluation</font>

In [28]:
'''Evaluate the model with R^2'''

# print the r2 score
from sklearn.metrics import r2_score 

r2_score(test_sentiment['sentiment'], test_reviews['RNN_Pred_Rating'])

KeyError: 'RNN_Pred_Rating'

On the full test dataset of 25,000 reviews, the $R^2$ for this model is 0.622525. So the RNN outperforms the continuous bag-of-words MLP as well as the standard bag-of-words approach.
Your score might not be good because we're training on only 100-200 reviews.

### <font color='#6629b2'>Visualizing data inside the model</font>

To help visualize the data representation inside the model, we can look at the output of each layer in a model individually. Keras' Functional API lets you derive a new model with the layers from an existing model, so you can define the output to be a layer below the output layer in the original model. Calling predict() on this new model will produce the output of that layer for a given input. Of course, glancing at the numbers by themselves doesn't provide any interpretation of what the model has learned (although there are opportunities to [interpret these values](https://medium.com/civis-analytics/interpreting-and-visualizing-neural-networks-for-text-processing-e9dff0da9c22), but seeing them verifies the model is just a series of transformations from one matrix to another. The model stores its layers as the list model.layers, and you can retrieve specific layer by its position index in the model.

In [17]:
'''Showing the output of the RNN embedding layer (second layer) for the test reviews'''

embedding_layer = Model(inputs=rnn_model.layers[0].input, 
                        outputs=rnn_model.layers[1].output) #embedding layer is 2nd layer (index 1)
embedding_output = embedding_layer.predict(test_padded_idxs)
print("EMBEDDING LAYER OUTPUT SHAPE:", embedding_output.shape)
print(embedding_output[0])

EMBEDDING LAYER OUTPUT SHAPE: (300, 1241, 500)
[[ 0.01040394  0.01428689 -0.02378903 ...  0.02953665  0.02730739
  -0.02176386]
 [-0.00468918  0.01375673 -0.0370001  ...  0.01716555  0.01333654
  -0.01692664]
 [-0.00660518  0.01213785 -0.04233654 ...  0.01695575  0.01011857
  -0.0147245 ]
 ...
 [-0.02498542  0.00779739 -0.08136724 ... -0.01197113  0.00325015
  -0.00482261]
 [-0.00907925 -0.02603161  0.00812179 ... -0.04037679  0.00179039
   0.02686844]
 [ 0.02090453 -0.02342772  0.06114694 ... -0.0108693   0.04019788
   0.00220405]]


## <font color='#6629b2'>Conclusion</font>

As mentioned above, the models shown here could be applied to any task where the goal is to predict a score for a particular sequence. For ratings prediction, this output is ordinal, but it could also be categorical with a few simple changes to the output layer of the model. 

## <font color='#6629b2'>More resources</font>

Yoav Goldberg's book [Neural Network Methods for Natural Language Processing](http://www.morganclaypool.com/doi/abs/10.2200/S00762ED1V01Y201703HLT037) is a thorough introduction to neural networks for NLP tasks in general.

If you'd like to learn more about what Keras is doing under the hood, there is a [Theano tutorial](http://deeplearning.net/tutorial/lstm.html) that also applies an RNN to sentiment prediction, using the same dataset here

Andrej Karpathy's blog post [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) is very helpful for understanding the mathematical details of an RNN, applied to the task of language modeling. It also provides raw Python code with an implementation of the backpropagation algorithm.

TensorFlow also has an RNN language model [tutorial](https://www.tensorflow.org/versions/r0.12/tutorials/recurrent/index.html) using the Penn Treebank dataset

Chris Olah provides a good [explanation](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) of how LSTM RNNs work (this explanation also applies to the GRU model used here)

Denny Britz's [tutorial](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) documents well both the technical details of RNNs and their implementation in Python.